In [1]:
# In this example we will show the difference between fixes computed with laika
# from raw data of the ublox receiver vs the the fixes the ublox receiver
# computes

In [2]:
import numpy as np
with open('example_data/raw_gnss_ublox/t', 'r') as f:
  raw_ublox_t = np.load(f)
with open('example_data/raw_gnss_ublox/value', 'r') as f:
  raw_ublox = np.load(f)
with open('example_data/live_gnss_ublox/t', 'r') as f:
  fixes_ublox_t = np.load(f)
with open('example_data/live_gnss_ublox/value', 'r') as f:
  fixes_ublox = np.load(f)

In [3]:
# We get the raw data into our format from the log array format

from laika.raw_gnss import normal_meas_from_array
measurements = np.array([normal_meas_from_array(arr) for arr in raw_ublox])

In [4]:
# initialize an astrodog with dgps corrections

from laika import AstroDog
dog = AstroDog(dgps=True)

In [5]:
# Building this cache takes forever just copy it from repo

from shutil import copyfile
import os
cache_directory = '/tmp/gnss/cors_coord/'
try:
  os.mkdir('/tmp/gnss/')
except:
  pass
try:
  os.mkdir(cache_directory)
except:
  pass
copyfile('cors_station_positions', cache_directory + 'cors_station_positions')

In [6]:
from laika.raw_gnss import process_measurements, correct_measurements, calc_pos_fix
from tqdm import tqdm

# We want to group measurements by measurement epoch
# this makes the kalman filter faster and is easier
# to reason about
grouped_t = sorted(list(set(raw_ublox_t)))                                                                                      
grouped_meas_processed = []
corrected_meas_arrays = []

# process measurement groups
for t in grouped_t:
  meas = measurements[raw_ublox_t == t]
  grouped_meas_processed.append(process_measurements(meas, dog))

# correct measurement groups with an estimate position
# that was computes with weighted-least-squares on
# the first epoch
# WARNING: can take up to 10min
wls_estimate = calc_pos_fix(grouped_meas_processed[0])
est_pos = wls_estimate[0][:3]
for proc in tqdm(grouped_meas_processed):
  corrected = correct_measurements(proc, est_pos, dog)
  corrected_meas_arrays.append(np.array([c.as_array() for c in corrected]))

No orbit data found for prn : G04 flagging as bad
No orbit data found for prn : R06 flagging as bad
No orbit data found for prn : R07 flagging as bad
No orbit data found for prn : R12 flagging as bad
No orbit data found for prn : R25 flagging as bad
No orbit data found for prn : R27 flagging as bad
No orbit data found for prn : R28 flagging as bad


  0%|          | 0/600 [00:00<?, ?it/s]

pulling from http://ftpcache.comma.life/geodesy-noaa-gov/cors/rinex/2018/214/pbl1/pbl12140.18o.gz to /tmp/gnss/cors_obs/2018/214/pbl1/pbl12140.18o
cache download failed, pulling from ftp://geodesy.noaa.gov/cors/rinex/2018/214/pbl1/pbl12140.18o.gz to /tmp/gnss/cors_obs/2018/214/pbl1/pbl12140.18o
pulling from http://ftpcache.comma.life/geodesy-noaa-gov/cors/rinex/2018/214/pbl2/pbl22140.18o.gz to /tmp/gnss/cors_obs/2018/214/pbl2/pbl22140.18o
cache download failed, pulling from ftp://geodesy.noaa.gov/cors/rinex/2018/214/pbl2/pbl22140.18o.gz to /tmp/gnss/cors_obs/2018/214/pbl2/pbl22140.18o
pulling from http://ftpcache.comma.life/geodesy-noaa-gov/cors/rinex/2018/214/hsib/hsib2140.18o.gz to /tmp/gnss/cors_obs/2018/214/hsib/hsib2140.18o
cache download failed, pulling from ftp://geodesy.noaa.gov/cors/rinex/2018/214/hsib/hsib2140.18o.gz to /tmp/gnss/cors_obs/2018/214/hsib/hsib2140.18o
No dcb data found for prn : R26 flagging as bad


100%|██████████| 600/600 [01:03<00:00,  9.48it/s] 


In [7]:
for proc in tqdm(grouped_meas_processed):
  corrected = correct_measurements(proc, est_pos, dog)
  corrected_meas_arrays.append(np.array([c.as_array() for c in corrected]))

100%|██████████| 600/600 [00:03<00:00, 166.33it/s]


In [9]:
# We run the kalman filter

from kalman.gnss_kf import GNSSKalman
from kalman.kalman_helpers import run_car_ekf_offline, ObservationKind
ekf = GNSSKalman()
init_state = ekf.x
init_state[:3] = est_pos
ekf.init_state(init_state)
ekf_data = {}
ekf_data[ObservationKind.PSEUDORANGE_GPS] = (grouped_t, corrected_meas_arrays)
ekf_data[ObservationKind.PSEUDORANGE_RATE_GPS] = (grouped_t, corrected_meas_arrays)
ekf_outputs = run_car_ekf_offline(ekf, ekf_data)

import laika.lib.coordinates as coord
laika_positions_t = ekf_outputs[4]
laika_positions_ecef = ekf_outputs[0][:,:3]
laika_positions_geodetic = coord.ecef2geodetic(laika_positions_ecef)

100%|██████████| 1200/1200 [00:00<00:00, 1473.30it/s]


In [10]:
ublox_positions_geodetic = fixes_ublox[:,[0,1,4]]

In [11]:
# By looking at the map, we can see that the two paths compared.
# If you want to regenerate the gmplot you will need a google
# maps API key

import gmplot
gmap = gmplot.GoogleMapPlotter(*laika_positions_geodetic[0])
#gmap.apikey='...'
gmap.plot([x[0]  for x in laika_positions_geodetic], [x[1] for x in laika_positions_geodetic], 'blue', edge_width = 5)
gmap.plot([x[0]  for x in ublox_positions_geodetic], [x[1] for x in ublox_positions_geodetic], 'red', edge_width = 5)
gmap.draw("laika_quality_check.html")



import webbrowser
import os
webbrowser.open('file://' + os.path.realpath("laika_quality_check.html"));